# Analisi dei dati raccolti

## Setup

In [1]:
import pandas as pd

In [2]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconxofv12

In [3]:
path = "../asconxofv12/data/{}.csv"

### Armv6m

In [4]:
with open(path.format("armv6m"), "r") as f:
    axof_armv6m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [5]:
# Generazione hash
data = axof_armv6m["0B"]

print_data(data)

0B
235    513
234    183
237     90
244     88
242     63
236     38
243     19
241      4
245      2
Name: count, dtype: int64

Massimo: 245
Minimo: 234
Media: 236.464
Varianza: 10.222926926926927


#### PT 8 byte

In [6]:
# Generazione hash
data = axof_armv6m["8B"]

print_data(data)

8B
288    414
289    177
290     98
297     85
298     74
296     63
295     61
291     26
299      2
Name: count, dtype: int64

Massimo: 299
Minimo: 288
Media: 290.909
Varianza: 13.902621621621623


#### PT 16 byte

In [7]:
# Generazione hash
data = axof_armv6m["16B"]

print_data(data)

16B
342    553
351    204
344     94
349     64
341     28
350     26
343     19
352     12
Name: count, dtype: int64

Massimo: 352
Minimo: 341
Media: 344.791
Varianza: 15.172491491491488


#### PT 32 byte

In [8]:
# Generazione hash
data = axof_armv6m["32B"]

print_data(data)

32B
449    411
458    221
451    120
459     88
450     66
456     56
457     36
452      2
Name: count, dtype: int64

Massimo: 459
Minimo: 449
Media: 452.861
Varianza: 17.4231021021021


#### PT 64 byte

In [9]:
# Generazione hash
data = axof_armv6m["64B"]

print_data(data)

64B
673    317
672    171
664    118
671     96
665     89
663     87
666     73
670     34
674     15
Name: count, dtype: int64

Massimo: 674
Minimo: 663
Media: 669.395
Varianza: 15.426401401401407


#### PT 128 byte

In [10]:
# Generazione hash
data = axof_armv6m["128B"]

print_data(data)

128B
1101    428
1102    383
1103     74
1104     40
1109     39
1111     14
1108     12
1110     10
Name: count, dtype: int64

Massimo: 1111
Minimo: 1101
Media: 1102.277
Varianza: 4.706977977977979


#### PT 256 byte

In [11]:
# Generazione hash
data = axof_armv6m["256B"]

print_data(data)

256B
1968    698
1969    249
1961     17
1967     16
1966     16
1959      3
1962      1
Name: count, dtype: int64

Massimo: 1969
Minimo: 1959
Media: 1968.049
Varianza: 1.440039039039039


#### PT 512 byte

In [12]:
# Generazione hash
data = axof_armv6m["512B"]

print_data(data)

512B
3702    388
3701    138
3693    108
3700    100
3695     78
3694     65
3703     51
3692     45
3699     18
3696      8
3691      1
Name: count, dtype: int64

Massimo: 3703
Minimo: 3691
Media: 3699.112
Varianza: 14.06351951951952


#### PT 1024 byte

In [13]:
# Generazione hash
data = axof_armv6m["1024B"]

print_data(data)

1024B
7160    479
7159    178
7161    135
7162     77
7167     55
7169     30
7168     22
7170     10
7158      5
7163      5
7166      4
Name: count, dtype: int64

Massimo: 7170
Minimo: 7158
Media: 7161.071
Varianza: 7.223182182182182


### Armv6m_lowsize

In [14]:
with open(path.format("armv6m_lowsize"), "r") as f:
    axof_lowsize = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [15]:
# Generazione hash
data = axof_lowsize["0B"]

print_data(data)

0B
239    386
240    358
238    148
237     98
245      4
241      3
244      2
246      1
Name: count, dtype: int64

Massimo: 246
Minimo: 237
Media: 239.061
Varianza: 1.1504294294294293


#### PT 8 byte

In [16]:
# Generazione hash
data = axof_lowsize["8B"]

print_data(data)

8B
300    500
299    285
301     59
294     58
292     43
293     29
302     13
295     13
Name: count, dtype: int64

Massimo: 302
Minimo: 292
Media: 298.84
Varianza: 5.642042042042043


#### PT 16 byte

In [17]:
# Generazione hash
data = axof_lowsize["16B"]

print_data(data)

16B
347    424
354    377
355    146
348     32
349     21
Name: count, dtype: int64

Massimo: 355
Minimo: 347
Media: 350.881
Varianza: 12.88372272272272


#### PT 32 byte

In [18]:
# Generazione hash
data = axof_lowsize["32B"]

print_data(data)

32B
457    267
458    258
456    239
459    176
464     38
463     19
466      2
465      1
Name: count, dtype: int64

Massimo: 466
Minimo: 456
Media: 457.777
Varianza: 3.3726436436436433


#### PT 64 byte

In [19]:
# Generazione hash
data = axof_lowsize["64B"]

print_data(data)

64B
677    535
678    251
683     57
682     38
675     37
676     29
685     27
684     26
Name: count, dtype: int64

Massimo: 685
Minimo: 675
Media: 678.078
Varianza: 5.275191191191191


#### PT 128 byte

In [20]:
# Generazione hash
data = axof_lowsize["128B"]

print_data(data)

128B
1123    276
1124    273
1122    263
1125    178
1130      4
1129      3
1126      2
1131      1
Name: count, dtype: int64

Massimo: 1131
Minimo: 1122
Media: 1123.426
Varianza: 1.4539779779779778


#### PT 256 byte

In [21]:
# Generazione hash
data = axof_lowsize["256B"]

print_data(data)

256B
2007    920
2008     80
Name: count, dtype: int64

Massimo: 2008
Minimo: 2007
Media: 2007.08
Varianza: 0.07367367367367365


#### PT 512 byte

In [22]:
# Generazione hash
data = axof_lowsize["512B"]

print_data(data)

512B
3777    360
3775    279
3776    236
3774     84
3778     39
3769      2
Name: count, dtype: int64

Massimo: 3778
Minimo: 3769
Media: 3775.977
Varianza: 1.2297007007007008


#### PT 1024 byte

In [23]:
# Generazione hash
data = axof_lowsize["1024B"]

print_data(data)

1024B
7307    611
7306    272
7308     77
7314     25
7313      8
7315      5
7309      2
Name: count, dtype: int64

Massimo: 7315
Minimo: 7306
Media: 7307.072
Varianza: 2.187003003003003


### Bi32_armv6m

In [24]:
with open(path.format("bi32_armv6m"), "r") as f:
    axof_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [25]:
# Generazione hash
data = axof_bi32["0B"]

print_data(data)

0B
240    401
239    193
247    175
242    138
241     55
246     35
248      2
243      1
Name: count, dtype: int64

Massimo: 248
Minimo: 239
Media: 241.592
Varianza: 8.245781781781783


#### PT 8 byte

In [26]:
# Generazione hash
data = axof_bi32["8B"]

print_data(data)

8B
294    339
301    312
296    201
293     75
295     57
302     16
Name: count, dtype: int64

Massimo: 302
Minimo: 293
Media: 296.696
Varianza: 9.989573573573573


#### PT 16 byte

In [27]:
# Generazione hash
data = axof_bi32["16B"]

print_data(data)

16B
350    339
348    312
355    246
356     77
349     25
357      1
Name: count, dtype: int64

Massimo: 357
Minimo: 348
Media: 351.05
Varianza: 9.150650650650649


#### PT 32 byte

In [28]:
# Generazione hash
data = axof_bi32["32B"]

print_data(data)

32B
458    353
464    220
463    180
459    122
456     77
457     36
465     12
Name: count, dtype: int64

Massimo: 465
Minimo: 456
Media: 460.236
Varianza: 8.482786786786788


#### PT 64 byte

In [29]:
# Generazione hash
data = axof_bi32["64B"]

print_data(data)

64B
675    356
682    267
680    215
683     80
681     59
674     23
Name: count, dtype: int64

Massimo: 683
Minimo: 674
Media: 678.915
Varianza: 10.408183183183182


#### PT 128 byte

In [30]:
# Generazione hash
data = axof_bi32["128B"]

print_data(data)

128B
1116    370
1115    347
1123     94
1117     90
1118     47
1122     36
1124     13
1119      2
1120      1
Name: count, dtype: int64

Massimo: 1124
Minimo: 1115
Media: 1116.825
Varianza: 6.719094094094096


#### PT 256 byte

In [31]:
# Generazione hash
data = axof_bi32["256B"]

print_data(data)

256B
1991    561
1992    369
1984     26
1989     18
1993     15
1990      8
1985      3
Name: count, dtype: int64

Massimo: 1993
Minimo: 1984
Media: 1991.155
Varianza: 1.8688438438438446


#### PT 512 byte

In [32]:
# Generazione hash
data = axof_bi32["512B"]

print_data(data)

512B
3743    426
3736    177
3741    158
3744     92
3742     69
3735     45
3737     31
3740      2
Name: count, dtype: int64

Massimo: 3744
Minimo: 3735
Media: 3740.916
Varianza: 9.146090090090093


#### PT 1024 byte

In [33]:
# Generazione hash
data = axof_bi32["1024B"]

print_data(data)

1024B
7237    314
7238    244
7239    153
7245    106
7240     94
7244     77
7246     11
7241      1
Name: count, dtype: int64

Massimo: 7246
Minimo: 7237
Media: 7239.322
Varianza: 7.782098098098096


### Ref

In [34]:
with open(path.format("ref"), "r") as f:
    axof_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [35]:
# Generazione hash
data = axof_ref["0B"]

print_data(data)

0B
386    238
388    167
387    132
394    128
393    108
395     80
396     72
389     70
397      5
Name: count, dtype: int64

Massimo: 397
Minimo: 386
Media: 389.951
Varianza: 13.602201201201199


#### PT 8 byte

In [36]:
# Generazione hash
data = axof_ref["8B"]

print_data(data)

8B
466    254
464    252
473    144
471    129
474     82
472     71
465     44
467     22
475      2
Name: count, dtype: int64

Massimo: 475
Minimo: 464
Media: 468.227
Varianza: 14.375846846846846


#### PT 16 byte

In [37]:
# Generazione hash
data = axof_ref["16B"]

print_data(data)

16B
544    249
551    228
549    201
542    170
552     59
550     56
543     28
541      9
Name: count, dtype: int64

Massimo: 552
Minimo: 541
Media: 547.014
Varianza: 13.707511511511507


#### PT 32 byte

In [38]:
# Generazione hash
data = axof_ref["32B"]

print_data(data)

32B
707    334
705    189
700    135
699    116
706    115
704     40
698     29
697     23
708     19
Name: count, dtype: int64

Massimo: 708
Minimo: 697
Media: 704.042
Varianza: 11.199435435435438


#### PT 64 byte

In [39]:
# Generazione hash
data = axof_ref["64B"]

print_data(data)

64B
1018    898
1019     54
1026     16
1020     14
1017     12
1025      3
1021      3
Name: count, dtype: int64

Massimo: 1026
Minimo: 1017
Media: 1018.228
Varianza: 1.269285285285285


#### PT 128 byte

In [40]:
# Generazione hash
data = axof_ref["128B"]

print_data(data)

128B
1650    250
1648    174
1642    141
1649    133
1643    117
1641     56
1647     54
1640     42
1651     28
1644      5
Name: count, dtype: int64

Massimo: 1651
Minimo: 1640
Media: 1646.484
Varianza: 12.666410410410412


#### PT 256 byte

In [41]:
# Generazione hash
data = axof_ref["256B"]

print_data(data)

256B
2904    617
2903    122
2905     89
2902     78
2897     58
2896     24
2901      7
2895      2
2898      2
2906      1
Name: count, dtype: int64

Massimo: 2906
Minimo: 2895
Media: 2903.164
Varianza: 4.507611611611611


#### PT 512 byte

In [42]:
# Generazione hash
data = axof_ref["512B"]

print_data(data)

512B
5412    210
5413    171
5414    150
5420    121
5422     92
5419     91
5421     86
5415     70
5423      9
Name: count, dtype: int64

Massimo: 5423
Minimo: 5412
Media: 5416.079
Varianza: 14.234993993993996


#### PT 1024 byte

In [43]:
# Generazione hash
data = axof_ref["1024B"]

print_data(data)

1024B
10446    202
10440    164
10439    152
10438    144
10441    131
10448     93
10445     71
10447     39
10449      2
10437      2
Name: count, dtype: int64

Massimo: 10449
Minimo: 10437
Media: 10442.287
Varianza: 12.731362362362361
